In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/train_titanic.csv")
df.head(10)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [38]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [39]:
df.shape

(891, 12)

In [40]:
# Check for missing values
print("Missing values before preprocessing:")
print(df.isnull().sum())


Missing values before preprocessing:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [41]:
# Bound / sanity checks
df = df[(df["Age"].isnull()) | ((df["Age"] > 0) & (df["Age"] <= 100))]
df = df[(df["Fare"] >= 0) & (df["Fare"] <= 600)]
df = df[df["Pclass"].isin([1, 2, 3])]
df = df[df["Embarked"].isnull() | df["Embarked"].isin(["C", "Q", "S"])]

In [42]:
# Fill missing values
df["Age"] = df["Age"].fillna(df["Age"].median())
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

In [43]:
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})
df["Embarked"] = df["Embarked"].map({"C": 0, "Q": 1, "S": 2})

In [44]:
print("Missing values after preprocessing:")
print(df.isnull().sum())

Missing values after preprocessing:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


In [45]:
df = df.drop(columns=["Cabin"])


In [46]:
# Correlation with target variable
corr = df.corr(numeric_only=True)["Survived"].sort_values(ascending=False)
print(corr)


Survived       1.000000
Sex            0.543351
Fare           0.257307
Parch          0.081629
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.064910
Embarked      -0.167675
Pclass        -0.338481
Name: Survived, dtype: float64


In [47]:
selected_features = ["Pclass", "Sex", "Age", "Fare", "Embarked"]


In [48]:
from sklearn.model_selection import train_test_split

X = df[selected_features]
y = df["Survived"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [53]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=42)

In [54]:
y_pred_lr = lr_model.predict(X_test_scaled)
print("===== Logistic Regression =====")
print(classification_report(y_test, y_pred_lr))

# Save model + scaler
joblib.dump(lr_model, "lr_titanic_model.pkl")
joblib.dump(scaler, "lr_scaler.pkl")

===== Logistic Regression =====
              precision    recall  f1-score   support

           0       0.80      0.84      0.82       110
           1       0.72      0.67      0.69        69

    accuracy                           0.77       179
   macro avg       0.76      0.75      0.75       179
weighted avg       0.77      0.77      0.77       179



['lr_scaler.pkl']

In [55]:
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [56]:
# Predict and evaluate
y_pred_rf = rf_model.predict(X_test)
print("===== Random Forest =====")
print(classification_report(y_test, y_pred_rf))

# Save model
joblib.dump(rf_model, "rf_titanic_model.pkl")

===== Random Forest =====
              precision    recall  f1-score   support

           0       0.85      0.88      0.87       110
           1       0.80      0.75      0.78        69

    accuracy                           0.83       179
   macro avg       0.83      0.82      0.82       179
weighted avg       0.83      0.83      0.83       179



['rf_titanic_model.pkl']

In [57]:
from sklearn.svm import SVC

# Use same scaled data as Logistic Regression
svm_model = SVC(random_state=42)
svm_model.fit(X_train_scaled, y_train)

SVC(random_state=42)

In [58]:
# Predict and evaluate
y_pred_svm = svm_model.predict(X_test_scaled)
print("===== SVM =====")
print(classification_report(y_test, y_pred_svm))

# Save model + scaler
joblib.dump(svm_model, "svm_titanic_model.pkl")
joblib.dump(scaler, "svm_scaler.pkl")

===== SVM =====
              precision    recall  f1-score   support

           0       0.79      0.95      0.86       110
           1       0.87      0.59      0.71        69

    accuracy                           0.81       179
   macro avg       0.83      0.77      0.78       179
weighted avg       0.82      0.81      0.80       179



['svm_scaler.pkl']

In [59]:
from sklearn.neighbors import KNeighborsClassifier

# Use same scaled data
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [60]:
# Predict and evaluate
y_pred_knn = knn_model.predict(X_test_scaled)
print("===== KNN =====")
print(classification_report(y_test, y_pred_knn))

# Save model + scaler
joblib.dump(knn_model, "knn_titanic_model.pkl")
joblib.dump(scaler, "knn_scaler.pkl")

===== KNN =====
              precision    recall  f1-score   support

           0       0.81      0.91      0.86       110
           1       0.82      0.67      0.74        69

    accuracy                           0.82       179
   macro avg       0.82      0.79      0.80       179
weighted avg       0.82      0.82      0.81       179



['knn_scaler.pkl']

In [61]:
from sklearn.svm import SVC

# Use same scaled data as Logistic Regression
svm_model = SVC(random_state=42)
svm_model.fit(X_train_scaled, y_train)

# Predict and evaluate
y_pred_svm = svm_model.predict(X_test_scaled)
print("===== SVM =====")
print(classification_report(y_test, y_pred_svm))

# Save model + scaler
joblib.dump(svm_model, "svm_titanic_model.pkl")
joblib.dump(scaler, "svm_scaler.pkl")


===== SVM =====
              precision    recall  f1-score   support

           0       0.79      0.95      0.86       110
           1       0.87      0.59      0.71        69

    accuracy                           0.81       179
   macro avg       0.83      0.77      0.78       179
weighted avg       0.82      0.81      0.80       179



['svm_scaler.pkl']

In [62]:
import joblib
joblib.dump(rf_model, "titanic_survival_model.pkl")


['titanic_survival_model.pkl']

# Random Forest is selected as the final model because it achieved the highest accuracy and F1-score among all evaluated models.

